In [1]:
import logging
import os
import random
import time
import warnings
warnings.filterwarnings("ignore")
from itertools import product
from multiprocessing import cpu_count, Pool # for prophet

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from prophet import Prophet
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, _TS
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.model_selection import ParameterGrid
from utilsforecast.plotting import plot_series

from importlib import reload
import utils
reload(utils)

from utils import prepare_data, TimeMoEPredictor, chronosPredictor, LLM, wape, ProphetForecast, fit_prophet, predict_prophet


from utilsforecast.losses import *

from functools import partial


from utilsforecast.evaluation import evaluate

Importing plotly failed. Interactive plots will not work.


In [28]:
import pandas as pd
from prophet import Prophet
from concurrent.futures import ProcessPoolExecutor

# Sample data with a 'group' column
data = pd.DataFrame({
    'ds': pd.date_range(start='2020-01-01', periods=100, freq='D'),
    'y': range(100),
    'group': ['A']*50 + ['B']*50
})

# Split data by the group column
groups = data['group'].unique()

# Prepare the data as a list of tuples (group_name, data)
group_data_list = [(group, data) for group in groups]

# Step 1: Fit Prophet models for each group in parallel
with ProcessPoolExecutor() as executor:
    fitted_models = list(executor.map(fit_prophet, group_data_list))

# Step 2: Make predictions using the fitted models in parallel
with ProcessPoolExecutor() as executor:
    predictions = list(executor.map(predict_prophet, fitted_models))

# Combine all prediction results into a single DataFrame
forecast_df = pd.concat([result[1] for result in predictions], ignore_index=True)

# Output forecast data
print(forecast_df.head())


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmpny1g1o_i/a4p3uzpi.json
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmpk9owa6ko/51v0uhlh.json
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmpny1g1o_i/kw98x5sn.json
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmpk9owa6ko/r2xmhfyq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdSta

          ds          yhat    yhat_lower    yhat_upper
0 2020-01-01 -4.403824e-18 -1.306085e-12  1.192471e-12
1 2020-01-02  1.000000e+00  1.000000e+00  1.000000e+00
2 2020-01-03  2.000000e+00  2.000000e+00  2.000000e+00
3 2020-01-04  3.000000e+00  3.000000e+00  3.000000e+00
4 2020-01-05  4.000000e+00  4.000000e+00  4.000000e+00


In [17]:
train, test = prepare_data(sample_size=1000,series_cutoff=48)

In [21]:
train

,unique_id,y,ds
0,M10000,2740.0,2013-04-30 12:00:00
1,M10000,3000.0,2013-05-31 12:00:00
2,M10000,3040.0,2013-06-30 12:00:00
3,M10000,2920.0,2013-07-31 12:00:00
4,M10000,2870.0,2013-08-31 12:00:00
...,...,...,...
46504,M9964,700.0,2016-08-31 12:00:00
46505,M9964,630.0,2016-09-30 12:00:00
46506,M9964,630.0,2016-10-31 12:00:00
46507,M9964,610.0,2016-11-30 12:00:00


In [18]:
pr = ProphetForecast( train,  weekly_seasonality=False, daily_seasonality=False, yearly_seasonality=True, interval_width=0.95 )

In [19]:
pr.fit(df=train )


DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmpxrpgdbab/4qh44yw5.json
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmpylogdxsq/cufjkxwu.json
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmpraj92esj/ypfssgcy.json
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmp_isa3ayc/9spbtvzk.json
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmprnj_udwv/4g1q8alh.json
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmpylogdxsq/wphs91_6.json
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmpxrpgdbab/09zoshnd.json
DEBUG:cmdstanpy:input tempfile: /var/folders/2z/r4rjm4611fx7hfd4yth093t40000gq/T/tmpraj92esj/43gwbr_y.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:running CmdStan, num_threads: Non

In [20]:
pr.predict(test,h=3)

Exception: Model has not been fit.

In [ ]:
pr.predict()

In [4]:
def fit(index, ts):
    df = ts.drop(columns='unique_id', axis=1)
    
    model = Prophet(
                    weekly_seasonality=False,
                    daily_seasonality=False,
                    yearly_seasonality=True,
                    interval_width=0.95
                    )
    
    model = model.fit(df)
    
    return model


In [5]:
init = time.time()
with Pool(cpu_count()) as pool:
    forecast_prophet = pool.starmap(fit, train.groupby('unique_id'))
end = time.time()
time_prophet = end - init
time_prophet


DEBUG:cmdstanpy:input tempfile: /tmp/tmpz1pz5ygr/2wdnbzfm.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9_34mno3/ppgymnc6.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpz1pz5ygr/d2x_uru6.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsc7_003v/x2u9adg1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4vro7tuy/dcd_gc4f.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqp8pp7jl/_4754dmc.json
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4g17ohql/opdpvvx4.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsc7_003v/bxx107im.json
DEBUG:cmdstanpy:CmdStan args: ['/home/jan/m4_tests/.venv/lib/python3.12/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=3161', 'data', 'file=/tmp/tmpz1pz5ygr/2wdnbzfm.json', 'init=/tmp/tmpz1pz5ygr/d2x_uru6.json', 'output', 'file=/tmp/tmpz1pz5ygr/prophet_modelb71cknhk/prophet_model-20241207162357.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
DEBUG:cmdstanpy:input tempfile: /tmp/tmpetir1y

23.729613542556763

In [6]:
def predict_prophet(model):
    fc = (model.make_future_dataframe(periods=3, 
                                            include_history=False, 
                                            freq='M'))
    
    return model.predict(fc)

with Pool(cpu_count()) as pool:
    forecasts = pool.map(predict_prophet, forecast_prophet)


In [45]:
forecasts[1]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2016-08-31 12:00:00,1098.204540,1089.162257,1107.283419,1098.137894,1098.277859,0.480173,0.480173,0.480173,0.480173,0.480173,0.480173,0.0,0.0,0.0,1098.684713
1,2016-09-30 12:00:00,1099.308169,1095.770897,1113.183372,1099.111307,1099.547465,5.255139,5.255139,5.255139,5.255139,5.255139,5.255139,0.0,0.0,0.0,1104.563308
2,2016-10-31 12:00:00,1100.448585,1112.806562,1130.585814,1100.086710,1100.864594,20.805982,20.805982,20.805982,20.805982,20.805982,20.805982,0.0,0.0,0.0,1121.254567


In [43]:
t = pd.concat([test[["unique_id","y"]].reset_index(),pd.concat(forecasts).reset_index()],axis=1).drop(columns="index").rename(columns = {"yhat":"prophet","yhat_upper":"Prophet-hi-95","yhat_lower":"Prophet-lo-95"})[["unique_id", "ds", "prophet","Prophet-hi-95","Prophet-lo-95"]]

In [44]:
t

,unique_id,ds,prophet,Prophet-hi-95,Prophet-lo-95
0,M10000,2017-01-31 12:00:00,1966.524438,2279.120937,1635.113515
1,M10000,2017-02-28 12:00:00,1839.142263,2156.017131,1515.919387
2,M10000,2017-03-31 12:00:00,2021.684233,2350.630715,1687.108309
3,M10127,2016-08-31 12:00:00,1098.684713,1107.283419,1089.162257
4,M10127,2016-09-30 12:00:00,1104.563308,1113.183372,1095.770897
...,...,...,...,...,...
2902,M9942,2017-02-28 12:00:00,1206.589198,1443.460753,999.423253
2903,M9942,2017-03-31 12:00:00,1242.711122,1474.877078,1013.777140
2904,M9964,2017-01-31 12:00:00,217.855938,373.027977,70.004623
2905,M9964,2017-02-28 12:00:00,311.606470,450.280272,165.061800


In [15]:
metrics = [
    mape,
    wape,
    rmse,
    partial(mase, seasonality=12),
    partial(rmsse, seasonality=12),
]

In [16]:
eval = evaluate(t, metrics=metrics, train_df=train)

In [17]:
metrics = eval.set_index(["unique_id","metric"]).stack().rename_axis(index={None: 'Model'}).rename("value")

In [18]:
metrics.groupby(level=[1,2]).median().unstack()

Model,prophet
metric,
mape,0.064713
mase,0.799822
rmse,262.093248
rmsse,0.714715
wape,0.062349


In [19]:
weights = (
    train
        .groupby("unique_id",)["y"]
        .sum()
        .rename("weights")
)

weights = weights/weights.sum()

In [20]:
(
    metrics[metrics.index.get_level_values(level=1).isin(["rmsse","rmse"])]
        .reset_index().merge(weights, on = "unique_id")
        .assign(metric = lambda x : "w"+x["metric"],
                value = lambda x : x.value*x.weights
                )
        .groupby(["metric","Model"])["value"]
        .sum()
        .sort_values()
)

metric  Model  
wrmsse  prophet      0.885253
wrmse   prophet    924.081849
Name: value, dtype: float64